In [ ]:
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error
from datetime import datetime, timedelta

In [2]:
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [16]:
def execute_query(connection, query, varlist=None):
    cursor = connection.cursor(buffered=True)
    try:
        if varlist is None:
            cursor.execute(query)
        else:
            cursor.execute(query, varlist)
        connection.commit()
        print("Query successful")
        return cursor.fetchall()
    except Error as err:
        print(f"Error: '{err}'")

def executemany_query(connection, query, varlist=None):
    cursor = connection.cursor()
    try:
        if varlist is None:
            cursor.executemany(query)
        else:
            cursor.executemany(query, varlist)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [17]:
pw = '27111998'
db = 'crypto_db'
connection = create_db_connection("localhost", "root", pw, db)

MySQL Database connection successful


In [42]:
query = 'SELECT * FROM btcusdt ORDER BY timestamp ASC'
res = execute_query(connection,query)
# cursor.execute(query)


Query successful


In [50]:
df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
df = df.drop_duplicates()
df

,timestamp,open,high,low,close,volume
0,1502942400000,4261.48,4313.62,4261.32,4308.83,47.181000
1,1502946000000,4308.83,4328.69,4291.37,4315.32,23.234900
2,1502949600000,4330.29,4345.45,4309.37,4324.35,7.229690
3,1502953200000,4316.62,4349.99,4287.41,4349.99,4.443250
4,1502956800000,4333.32,4377.85,4333.32,4360.69,0.972807
...,...,...,...,...,...,...
56932,1708156800000,51911.30,51911.30,51702.40,51740.00,837.773000
56933,1708160400000,51740.00,51795.80,51651.50,51676.40,931.136000
56934,1708164000000,51676.40,51729.20,51500.00,51721.70,1322.860000
56935,1708167600000,51721.70,51800.00,51677.20,51690.50,719.365000


In [54]:
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.head(25)

,timestamp,open,high,low,close,volume
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181000
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234900
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229690
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443250
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807
5,2017-08-17 09:00:00,4360.00,4445.78,4360.00,4444.00,10.763600
6,2017-08-17 10:00:00,4441.10,4470.00,4399.81,4460.00,24.865400
7,2017-08-17 11:00:00,4460.00,4485.39,4427.30,4427.30,27.018100
8,2017-08-17 12:00:00,4436.06,4449.56,4411.00,4411.00,23.032400
9,2017-08-17 13:00:00,4411.00,4459.00,4411.00,4459.00,31.312400


In [83]:
day_df = (df.resample('D', on='timestamp')
               .agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'}))
day_df

,open,high,low,close
timestamp,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08
2017-08-18,4285.08,4371.52,3938.77,4108.37
2017-08-19,4108.37,4184.69,3850.00,4139.98
2017-08-20,4120.98,4211.08,4032.62,4086.29
2017-08-21,4069.13,4119.62,3911.79,4016.00
...,...,...,...,...
2024-02-13,49917.30,50368.60,48300.90,49699.60
2024-02-14,49699.60,52043.70,49225.00,51795.20
2024-02-15,51795.20,52816.60,51314.00,51880.00


In [84]:
day_df['timestamp'] = day_df.index
day_df

,open,high,low,close,timestamp
timestamp,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,2017-08-17
2017-08-18,4285.08,4371.52,3938.77,4108.37,2017-08-18
2017-08-19,4108.37,4184.69,3850.00,4139.98,2017-08-19
2017-08-20,4120.98,4211.08,4032.62,4086.29,2017-08-20
2017-08-21,4069.13,4119.62,3911.79,4016.00,2017-08-21
...,...,...,...,...,...
2024-02-13,49917.30,50368.60,48300.90,49699.60,2024-02-13
2024-02-14,49699.60,52043.70,49225.00,51795.20,2024-02-14
2024-02-15,51795.20,52816.60,51314.00,51880.00,2024-02-15


In [85]:
PUMP_THRESHOLD = 1
PUMP_THRESHOLD_INITIAL = 0.5
WINDOW_BEFORE = 7
WINDOW_AFTER = 14
PCT_THRESHOLD = 0.3

data_pump = []
max_idx = max(day_df.index)
# df_checked_dropna = df_checked[i].dropna()
j = list(day_df.index)[0]
while j < max_idx:
    try:
        is_pump = False
        low_bound = j
        up_bound = min([low_bound+pd.DateOffset(days=WINDOW_AFTER), max_idx+pd.DateOffset(days=1)])
        df_temp_next = day_df.loc[low_bound:up_bound]
        max_price = max(df_temp_next['close'])
        idx_max_price = list(df_temp_next.index)[df_temp_next['close'].argmax()]
        # print(idx_max_price)
        base_price = np.percentile(df_temp_next['close'],10)
        idx_base_price = list(df_temp_next.index)[df_temp_next['close'].argmin()]
        pct_to_max = (max_price - base_price)/base_price
        if  pct_to_max > PUMP_THRESHOLD_INITIAL:
            pump_analysis = True
        else:
            pump_analysis = False
            j = j+pd.DateOffset(days=1)
        while pump_analysis:
            low_bound = idx_max_price + pd.DateOffset(days=1)
            up_bound = min([low_bound+pd.DateOffset(days=WINDOW_AFTER), max_idx+pd.DateOffset(days=1)])
            df_temp_next = day_df.loc[low_bound:up_bound]
            max_price_new = max(df_temp_next['close'])
            idx_max_price_new = list(df_temp_next.index)[df_temp_next['close'].argmax()]
            pct_to_max_new = (max_price_new - base_price)/base_price
            if pct_to_max_new > PUMP_THRESHOLD and max_price_new > max_price:
                idx_max_price = idx_max_price_new
                max_price = max_price_new
                pct_to_max = pct_to_max_new
                is_pump = True
            else:
                pump_analysis = False
                if is_pump:
                    data_pump.append(['btcusdt', day_df['timestamp'].loc[idx_base_price], base_price, day_df['timestamp'].loc[idx_max_price], max_price, pct_to_max])
                j = idx_max_price + pd.DateOffset(days=1)
    except ValueError: j = j+pd.DateOffset(days=1)
    except Exception as e: 
        print(e)
        break

In [86]:
data_pump

[['btcusdt',
  Timestamp('2017-11-23 00:00:00'),
  8165.168,
  Timestamp('2017-12-16 00:00:00'),
  19102.7,
  1.3395354510770632]]

ALL MARKET

In [87]:
import ccxt
exchange_id = 'tokocrypto'
exchange_class = getattr(ccxt, exchange_id)
exchange = exchange_class({
    'apiKey': 'YOUR_API_KEY',
    'secret': 'YOUR_SECRET',
})
markets = exchange.load_markets()
market_list = list(markets.keys())
considered_market = []
for i in market_list:
    if '/USDT' in i or '/BTC' in i:
        considered_market.append(i)

print(len(considered_market))
print(considered_market)

605
['1INCH/BTC', '1INCH/USDT', 'AAVE/USDT', 'ACA/BTC', 'ACA/USDT', 'ACE/BTC', 'ACE/USDT', 'ACH/BTC', 'ACH/USDT', 'ACM/USDT', 'ADA/BTC', 'ADA/USDT', 'ADX/BTC', 'ADX/USDT', 'AERGO/BTC', 'AERGO/USDT', 'AGIX/BTC', 'AGIX/USDT', 'AGLD/BTC', 'AGLD/USDT', 'AI/BTC', 'AI/USDT', 'AKRO/USDT', 'ALCX/BTC', 'ALCX/USDT', 'ALGO/BTC', 'ALGO/USDT', 'ALICE/BTC', 'ALICE/USDT', 'ALPACA/BTC', 'ALPACA/USDT', 'ALPHA/BTC', 'ALPHA/USDT', 'ALPINE/BTC', 'ALPINE/USDT', 'ALT/BTC', 'ALT/USDT', 'AMP/USDT', 'ANKR/BTC', 'ANKR/USDT', 'ANT/BTC', 'ANT/USDT', 'APE/BTC', 'APE/USDT', 'API3/BTC', 'API3/USDT', 'APT/BTC', 'APT/USDT', 'ARB/BTC', 'ARB/USDT', 'ARKM/BTC', 'ARKM/USDT', 'ARPA/BTC', 'ARPA/USDT', 'AR/BTC', 'AR/USDT', 'ASR/USDT', 'ASTR/USDT', 'AST/BTC', 'ATA/BTC', 'ATA/USDT', 'ATM/USDT', 'ATOM/BTC', 'ATOM/USDT', 'AUCTION/BTC', 'AUCTION/USDT', 'AUDIO/BTC', 'AUDIO/USDT', 'AVAX/BTC', 'AVAX/USDT', 'AVA/BTC', 'AVA/USDT', 'AXS/BTC', 'AXS/USDT', 'BADGER/BTC', 'BADGER/USDT', 'BAL/BTC', 'BAL/USDT', 'BAND/BTC', 'BAND/USDT', 'BAR/

In [94]:
PUMP_THRESHOLD = 1
PUMP_THRESHOLD_INITIAL = 0.5
WINDOW_BEFORE = 7
WINDOW_AFTER = 14
PCT_THRESHOLD = 0.3
data_pump = []
for i in range(len(considered_market)):
    try:
        market = considered_market[i]
        market = market.replace('/', '')
        print(market)
        query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
        res = execute_query(connection,query)
        df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df = df.drop_duplicates()
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'}))
        day_df['timestamp'] = day_df.index

        max_idx = max(day_df.index)
        # df_checked_dropna = df_checked[i].dropna()
        j = list(day_df.index)[0]
        while j < max_idx:
            try:
                is_pump = False
                low_bound = j
                up_bound = min([low_bound+pd.DateOffset(days=WINDOW_AFTER), max_idx+pd.DateOffset(days=1)])
                df_temp_next = day_df.loc[low_bound:up_bound]
                max_price = max(df_temp_next['close'])
                idx_max_price = list(df_temp_next.index)[df_temp_next['close'].argmax()]
                # print(idx_max_price)
                base_price = np.percentile(df_temp_next['close'],10)
                idx_base_price = list(df_temp_next.index)[df_temp_next['close'].argmin()]
                pct_to_max = (max_price - base_price)/base_price
                if  pct_to_max > PUMP_THRESHOLD_INITIAL:
                    pump_analysis = True
                else:
                    pump_analysis = False
                    j = j+pd.DateOffset(days=1)
                while pump_analysis:
                    low_bound = idx_max_price + pd.DateOffset(days=1)
                    up_bound = min([low_bound+pd.DateOffset(days=WINDOW_AFTER), max_idx+pd.DateOffset(days=1)])
                    df_temp_next = day_df.loc[low_bound:up_bound]
                    max_price_new = max(df_temp_next['close'])
                    idx_max_price_new = list(df_temp_next.index)[df_temp_next['close'].argmax()]
                    pct_to_max_new = (max_price_new - base_price)/base_price
                    if pct_to_max_new > PUMP_THRESHOLD and max_price_new > max_price:
                        idx_max_price = idx_max_price_new
                        max_price = max_price_new
                        pct_to_max = pct_to_max_new
                        is_pump = True
                    else:
                        pump_analysis = False
                        if is_pump:
                            data_pump.append([market, day_df['timestamp'].loc[idx_base_price], base_price, day_df['timestamp'].loc[idx_max_price], max_price, pct_to_max])
                        j = idx_max_price + pd.DateOffset(days=1)
            except ValueError: j = j+pd.DateOffset(days=1)
            except Exception as e: 
                print(e)
                break
    except Exception as e:
        print('market failed: ', market, ' error: ', e)
        continue




1INCHBTC
Query successful
1INCHUSDT
Query successful
AAVEUSDT
Query successful
ACABTC
Query successful
ACAUSDT
Query successful
ACEBTC
Query successful
ACEUSDT
Query successful
ACHBTC
Query successful
ACHUSDT
Query successful
ACMUSDT
Query successful
ADABTC
Query successful
ADAUSDT
Query successful
ADXBTC
Query successful
ADXUSDT
Query successful
AERGOBTC
Query successful
AERGOUSDT
Query successful
AGIXBTC
Query successful
AGIXUSDT
Query successful
AGLDBTC
Query successful
AGLDUSDT
Query successful
AIBTC
Query successful
AIUSDT
Query successful
AKROUSDT
Query successful
ALCXBTC
Query successful
ALCXUSDT
Query successful
ALGOBTC
Query successful
ALGOUSDT
Query successful
ALICEBTC
Query successful
ALICEUSDT
Query successful
ALPACABTC
Query successful
ALPACAUSDT
Query successful
ALPHABTC
Query successful
ALPHAUSDT
Query successful
ALPINEBTC
Query successful
ALPINEUSDT
Query successful
ALTBTC
Query successful
ALTUSDT
Query successful
AMPUSDT
Query successful
ANKRBTC
Query successful
ANKRUS

In [99]:
print(len(data_pump))
data_pump[2]

1246


['AAVEUSDT',
 Timestamp('2020-11-04 00:00:00'),
 28.5708,
 Timestamp('2020-12-05 00:00:00'),
 92.714,
 2.24506139135061]

In [100]:
df_data_pump = pd.DataFrame(data_pump, columns=['market','start','base_price','end','last_price','increase'])

In [104]:
dict_initial_listing = {}
for i in range(len(considered_market)):
    try:
        market = considered_market[i]
        market = market.replace('/', '')
        print(market)
        query = 'SELECT * FROM %s ORDER BY timestamp ASC' % market
        res = execute_query(connection,query)
        df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
        df = df.drop_duplicates()
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
        day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last'}))
        day_df['timestamp'] = day_df.index
        dict_initial_listing[market] = day_df['timestamp'].iloc[0]
    except:
        continue
        

1INCHBTC
Query successful
1INCHUSDT
Query successful
AAVEUSDT
Query successful
ACABTC
Query successful
ACAUSDT
Query successful
ACEBTC
Query successful
ACEUSDT
Query successful
ACHBTC
Query successful
ACHUSDT
Query successful
ACMUSDT
Query successful
ADABTC
Query successful
ADAUSDT
Query successful
ADXBTC
Query successful
ADXUSDT
Query successful
AERGOBTC
Query successful
AERGOUSDT
Query successful
AGIXBTC
Query successful
AGIXUSDT
Query successful
AGLDBTC
Query successful
AGLDUSDT
Query successful
AIBTC
Query successful
AIUSDT
Query successful
AKROUSDT
Query successful
ALCXBTC
Query successful
ALCXUSDT
Query successful
ALGOBTC
Query successful
ALGOUSDT
Query successful
ALICEBTC
Query successful
ALICEUSDT
Query successful
ALPACABTC
Query successful
ALPACAUSDT
Query successful
ALPHABTC
Query successful
ALPHAUSDT
Query successful
ALPINEBTC
Query successful
ALPINEUSDT
Query successful
ALTBTC
Query successful
ALTUSDT
Query successful
AMPUSDT
Query successful
ANKRBTC
Query successful
ANKRUS

In [105]:
dict_initial_listing

{'1INCHBTC': Timestamp('2020-12-25 00:00:00'),
 '1INCHUSDT': Timestamp('2020-12-25 00:00:00'),
 'AAVEUSDT': Timestamp('2020-10-15 00:00:00'),
 'ACABTC': Timestamp('2022-01-25 00:00:00'),
 'ACAUSDT': Timestamp('2022-01-25 00:00:00'),
 'ACEBTC': Timestamp('2023-12-18 00:00:00'),
 'ACEUSDT': Timestamp('2023-12-18 00:00:00'),
 'ACHBTC': Timestamp('2022-01-10 00:00:00'),
 'ACHUSDT': Timestamp('2022-01-10 00:00:00'),
 'ACMUSDT': Timestamp('2021-02-24 00:00:00'),
 'ADABTC': Timestamp('2017-11-30 00:00:00'),
 'ADAUSDT': Timestamp('2018-04-17 00:00:00'),
 'ADXBTC': Timestamp('2017-11-29 00:00:00'),
 'ADXUSDT': Timestamp('2021-10-29 00:00:00'),
 'AERGOBTC': Timestamp('2020-10-21 00:00:00'),
 'AERGOUSDT': Timestamp('2023-05-05 00:00:00'),
 'AGIXBTC': Timestamp('2021-06-02 00:00:00'),
 'AGIXUSDT': Timestamp('2023-02-17 00:00:00'),
 'AGLDBTC': Timestamp('2021-10-05 00:00:00'),
 'AGLDUSDT': Timestamp('2021-10-05 00:00:00'),
 'AIBTC': Timestamp('2024-01-04 00:00:00'),
 'AIUSDT': Timestamp('2024-01-04

In [107]:
df_data_pump['pump_length'] = df_data_pump['end'] - df_data_pump['start']
list_market_to_listing = [dict_initial_listing[i] for i in df_data_pump['market']]
df_data_pump['date_listing'] = list_market_to_listing
df_data_pump['pump_to_listing'] = df_data_pump['start'] - df_data_pump['date_listing']
df_data_pump

,market,start,base_price,end,last_price,increase,pump_length,date_listing,pump_to_listing
0,1INCHBTC,2021-01-08,0.000031,2021-02-05,0.000152,3.881960,28 days,2020-12-25,14 days
1,1INCHUSDT,2021-01-02,1.062220,2021-02-05,5.825000,4.483798,34 days,2020-12-25,8 days
2,AAVEUSDT,2020-11-04,28.570800,2020-12-05,92.714000,2.245061,31 days,2020-10-15,20 days
3,AAVEUSDT,2020-12-26,76.131200,2021-02-12,528.325000,5.939665,48 days,2020-10-15,72 days
4,ACAUSDT,2023-06-10,0.032340,2023-07-12,0.075700,1.340754,32 days,2022-01-25,501 days
...,...,...,...,...,...,...,...,...,...
1241,ZRXBTC,2017-12-11,0.000015,2018-01-27,0.000184,11.382234,47 days,2017-09-13,89 days
1242,ZRXBTC,2021-01-21,0.000016,2021-02-05,0.000041,1.632614,15 days,2017-09-13,1226 days
1243,ZRXUSDT,2020-04-23,0.184960,2020-05-09,0.422600,1.284818,16 days,2019-02-28,420 days
1244,ZRXUSDT,2021-01-21,0.514640,2021-02-13,1.891200,2.674802,23 days,2019-02-28,693 days


In [108]:
df_data_pump.to_csv('data_pump.csv')

In [112]:
df_data_pump_not_ico = df_data_pump[df_data_pump['pump_to_listing'] > timedelta(days=30)]
df_data_pump_not_ico

,market,start,base_price,end,last_price,increase,pump_length,date_listing,pump_to_listing
3,AAVEUSDT,2020-12-26,7.613120e+01,2021-02-12,528.325000,5.939665,48 days,2020-10-15,72 days
4,ACAUSDT,2023-06-10,3.234000e-02,2023-07-12,0.075700,1.340754,32 days,2022-01-25,501 days
5,ACHBTC,2023-01-24,4.800000e-07,2023-02-23,0.000002,2.979167,30 days,2022-01-10,379 days
6,ACHUSDT,2023-01-24,1.090800e-02,2023-02-23,0.045660,3.185919,30 days,2022-01-10,379 days
8,ADABTC,2021-01-29,1.028400e-05,2021-02-28,0.000029,1.828666,30 days,2017-11-30,1156 days
...,...,...,...,...,...,...,...,...,...
1241,ZRXBTC,2017-12-11,1.486000e-05,2018-01-27,0.000184,11.382234,47 days,2017-09-13,89 days
1242,ZRXBTC,2021-01-21,1.576000e-05,2021-02-05,0.000041,1.632614,15 days,2017-09-13,1226 days
1243,ZRXUSDT,2020-04-23,1.849600e-01,2020-05-09,0.422600,1.284818,16 days,2019-02-28,420 days
1244,ZRXUSDT,2021-01-21,5.146400e-01,2021-02-13,1.891200,2.674802,23 days,2019-02-28,693 days


In [137]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1)

# fig = go.Figure()

query = 'SELECT * FROM btcusdt ORDER BY timestamp ASC'
res = execute_query(connection,query)
df = pd.DataFrame(res,columns=['timestamp', 'open','high', 'low','close','volume'])
df = df.drop_duplicates()
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
day_df = (df.resample('D', on='timestamp').agg({'open': 'first', 'high': 'max', 'low': 'min', 'close': 'last','volume': 'sum'}))
day_df['timestamp'] = day_df.index

df_btcusdt = day_df.copy()
df_btcusdt_h = df.copy()

for i in df_data_pump_not_ico.index:
    fig.add_trace(go.Scatter(x=[df_data_pump_not_ico['start'][i], df_data_pump_not_ico['end'][i]], y=[df_data_pump_not_ico['market'][i],df_data_pump_not_ico['market'][i]],
                        mode='markers+lines'),row=1, col=1)
    
fig.add_trace(go.Scatter(x=df_btcusdt['timestamp'], y=df_btcusdt['close'], mode='markers+lines'),row=2, col=1)
fig.update_layout(height=600, width=1200)
# fig = px.line(x=[df_data_pump_not_ico['start'][8], df_data_pump_not_ico['end'][8]],y=[df_data_pump_not_ico['market'][8],df_data_pump_not_ico['market'][8]])
fig.show()

Query successful


In [128]:
df_btcusdt

,open,high,low,close,volume,timestamp
timestamp,,,,,,
2017-08-17,4261.48,4485.39,4200.74,4285.08,795.150347,2017-08-17
2017-08-18,4285.08,4371.52,3938.77,4108.37,1199.888200,2017-08-18
2017-08-19,4108.37,4184.69,3850.00,4139.98,381.309799,2017-08-19
2017-08-20,4120.98,4211.08,4032.62,4086.29,467.083144,2017-08-20
2017-08-21,4069.13,4119.62,3911.79,4016.00,691.742950,2017-08-21
...,...,...,...,...,...,...
2024-02-13,49917.30,50368.60,48300.90,49699.60,55551.565000,2024-02-13
2024-02-14,49699.60,52043.70,49225.00,51795.20,57046.394000,2024-02-14
2024-02-15,51795.20,52816.60,51314.00,51880.00,53816.040000,2024-02-15


In [125]:
df_data_pump_not_ico

,market,start,base_price,end,last_price,increase,pump_length,date_listing,pump_to_listing
3,AAVEUSDT,2020-12-26,7.613120e+01,2021-02-12,528.325000,5.939665,48 days,2020-10-15,72 days
4,ACAUSDT,2023-06-10,3.234000e-02,2023-07-12,0.075700,1.340754,32 days,2022-01-25,501 days
5,ACHBTC,2023-01-24,4.800000e-07,2023-02-23,0.000002,2.979167,30 days,2022-01-10,379 days
6,ACHUSDT,2023-01-24,1.090800e-02,2023-02-23,0.045660,3.185919,30 days,2022-01-10,379 days
8,ADABTC,2021-01-29,1.028400e-05,2021-02-28,0.000029,1.828666,30 days,2017-11-30,1156 days
...,...,...,...,...,...,...,...,...,...
1241,ZRXBTC,2017-12-11,1.486000e-05,2018-01-27,0.000184,11.382234,47 days,2017-09-13,89 days
1242,ZRXBTC,2021-01-21,1.576000e-05,2021-02-05,0.000041,1.632614,15 days,2017-09-13,1226 days
1243,ZRXUSDT,2020-04-23,1.849600e-01,2020-05-09,0.422600,1.284818,16 days,2019-02-28,420 days
1244,ZRXUSDT,2021-01-21,5.146400e-01,2021-02-13,1.891200,2.674802,23 days,2019-02-28,693 days


In [140]:
from ta import add_all_ta_features


df = add_all_ta_features(
    df_btcusdt_h, open="open", high="high", low="low", close="close", volume="volume", fillna=True)

In [141]:
df

,timestamp,open,high,low,close,volume,volume_adi,volume_obv,volume_cmf,volume_fi,...,momentum_ppo,momentum_ppo_signal,momentum_ppo_hist,momentum_pvo,momentum_pvo_signal,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr
0,2017-08-17 04:00:00,4261.48,4313.62,4261.32,4308.83,47.181000,3.853867e+01,4.718100e+01,0.816826,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4308.830000,0.000000,0.000000,0.000000
1,2017-08-17 05:00:00,4308.83,4328.69,4291.37,4315.32,23.234900,4.512562e+01,7.041590e+01,0.640844,150.794501,...,0.012014,0.002403,0.009611,-4.206887,-0.841377,-3.365510,4311.748390,0.150621,0.150508,0.150621
2,2017-08-17 06:00:00,4330.29,4345.45,4309.37,4324.35,7.229690,4.389930e+01,7.764559e+01,0.565381,138.578730,...,0.037996,0.009521,0.028474,-10.948641,-2.862830,-8.085811,4317.391270,0.209254,0.209036,0.360191
3,2017-08-17 07:00:00,4316.62,4349.99,4287.41,4349.99,4.443250,4.834255e+01,8.208884e+01,0.588905,135.056758,...,0.105301,0.028677,0.076624,-17.575002,-5.805264,-11.769737,4331.963869,0.592921,0.591171,0.955248
4,2017-08-17 08:00:00,4333.32,4377.85,4333.32,4360.69,0.972807,4.856560e+01,8.306165e+01,0.584693,117.249941,...,0.176499,0.058242,0.118257,-24.417609,-9.527733,-14.889876,4344.787882,0.245978,0.245676,1.203575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56932,2024-02-17 08:00:00,51911.30,51911.30,51702.40,51740.00,837.773000,6.091926e+06,1.016451e+06,0.149378,-34494.156123,...,-0.048980,-0.002524,-0.046456,-34.974010,-31.819334,-3.154677,51895.511703,-0.329986,-0.330532,1100.790006
56933,2024-02-17 09:00:00,51740.00,51795.80,51651.50,51676.40,931.136000,6.091317e+06,1.015520e+06,0.041401,-38026.455191,...,-0.083761,-0.018771,-0.064989,-30.695292,-31.594525,0.899234,51839.678083,-0.122922,-0.122998,1099.313967
56934,2024-02-17 10:00:00,51676.40,51729.20,51500.00,51721.70,1322.860000,6.092553e+06,1.016843e+06,0.243949,-24033.310449,...,-0.103113,-0.035640,-0.067473,-23.274462,-29.930512,6.656051,51822.582390,0.087661,0.087623,1100.365296
56935,2024-02-17 11:00:00,51721.70,51800.00,51677.20,51690.50,719.365000,6.091989e+06,1.016123e+06,0.088641,-23806.292957,...,-0.121914,-0.052895,-0.069019,-22.776470,-28.499704,5.723234,51803.512593,-0.060323,-0.060341,1099.641202


In [143]:
import ta
macd = ta.trend.MACD(df_btcusdt['close'])

In [144]:
macd

In [145]:
macd.macd()

timestamp
2017-08-17            NaN
2017-08-18            NaN
2017-08-19            NaN
2017-08-20            NaN
2017-08-21            NaN
                 ...     
2024-02-13    1687.648428
2024-02-14    1993.467049
2024-02-15    2217.115631
2024-02-16    2386.544969
2024-02-17    2457.526417
Freq: D, Name: MACD_12_26, Length: 2376, dtype: float64

In [147]:
macd.macd_diff()

timestamp
2017-08-17           NaN
2017-08-18           NaN
2017-08-19           NaN
2017-08-20           NaN
2017-08-21           NaN
                 ...    
2024-02-13    781.479671
2024-02-14    869.838633
2024-02-15    874.789773
2024-02-16    835.375289
2024-02-17    725.085389
Freq: D, Name: MACD_diff_12_26, Length: 2376, dtype: float64

In [148]:
macd.macd_signal()

timestamp
2017-08-17            NaN
2017-08-18            NaN
2017-08-19            NaN
2017-08-20            NaN
2017-08-21            NaN
                 ...     
2024-02-13     906.168757
2024-02-14    1123.628416
2024-02-15    1342.325859
2024-02-16    1551.169681
2024-02-17    1732.441028
Freq: D, Name: MACD_sign_12_26, Length: 2376, dtype: float64